In [ ]:
!pip install anvil-uplink
import anvil.server
import anvil.media
anvil.server.connect("3Q3GVRPPVXBL77VRBID5YJV6-JV7P4UGO7KJ72DNA")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
import os
import pickle
import wandb

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (32,32,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(units = 256, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))


model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   brightness_range=[0.6,1.0],
                                   width_shift_range=[-0.1,0.1])

test_datagen = ImageDataGenerator(rescale = 1./255)
#                                  shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    brightness_range=[0.6,1.0]
    

train_generator = train_datagen.flow_from_directory(
    directory = '../input/digit-test-18/Digits/Training',
    target_size = (32,32),
    batch_size = 16,
    class_mode = 'categorical'
)

test_generator = test_datagen.flow_from_directory(
    directory = '../input/digit-test-18/Digits/Validation',
    target_size = (32,32),
    batch_size = 16,
    class_mode = 'categorical',
    shuffle=False
)

train_steps_per_epoch = np.math.ceil(train_generator.samples / train_generator.batch_size)
print (train_steps_per_epoch)

test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
print (test_steps_per_epoch)


In [ ]:
history = model.fit(train_generator,
                         steps_per_epoch =train_steps_per_epoch,
                         epochs = 25,
                         shuffle = True,
                         validation_data =test_generator,
                         validation_steps = test_steps_per_epoch)

In [ ]:
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('training and validation accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()    
    
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('training and validation loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()  

In [ ]:
predictions = model.predict(test_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)
print(predicted_classes)

In [ ]:
true_classes = test_generator.classes
print(true_classes)

In [ ]:
class_labels = list(test_generator.class_indices.keys())
print(class_labels)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
conf_mat = confusion_matrix(true_classes, predicted_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat,
                              display_labels=class_labels)
   
disp.plot() 

In [ ]:
report=classification_report(true_classes, predicted_classes, target_names=class_labels,output_dict=True)
df = pd.DataFrame(report).transpose()
df

In [ ]:
from IPython.display import HTML
html = df.to_html()
# write html to file
text_file = open("index.html", "w")
text_file.write(html)
text_file.close()

In [ ]:
with wandb.init(project="img_digit_classifier",save_code=True) as run: 
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  #plt.show()    
  wandb.log({"Accuracy-metric": plt}) 
   
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  #plt.show()

  wandb.log({"Loss-metric": plt})

  model.save(os.path.join(wandb.run.dir, "model.h5"))
  
   
  disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat,display_labels=class_labels) 
  disp=disp.plot()
  wandb.log({"conf_mat" : plt})

  wandb.log({"Classification_report": wandb.Html(open("./index.html"))})
wandb.finish()

In [ ]:
model.save("cnn_model.h5")

In [ ]:
model = keras.models.load_model("cnn_model.h5")

In [ ]:
def get_result(result):
    if result[0][0] == 1:
        return('0')
    elif result[0][1] == 1:
        return ('1')
    elif result[0][2] == 1:
        return ('2')
    elif result[0][3] == 1:
        return ('3')
    elif result[0][4] == 1:
        return ('4')
    elif result[0][5] == 1:
        return ('5')
    elif result[0][6] == 1:
        return ('6')
    elif result[0][7] == 1:
        return ('7')
    elif result[0][8] == 1:
        return ('8')
    elif result[0][9] == 1:
        return ('9')

In [ ]:
filename = r'../input/digit-data-final/Dataset_2/Testing_2/8/download (1).jpeg'
test_image = image.load_img(filename, target_size = (32,32))
plt.imshow(test_image)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

In [ ]:
result = model.predict(test_image)
result = get_result(result)
print ('Predicted Alphabet is: {}'.format(result))

In [ ]:
@anvil.server.callable
def model_run_digit(path):
    with anvil.media.TempFile(path) as filename:
        test_image = image.load_img(filename, target_size = (32,32))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = model.predict(test_image)
        result = get_result(result)
        return ('Predicted Alphabet is: {}'.format(result))
        